In [18]:
# Install required packages
!pip install pandas deep-translator langdetect nltk Sastrawi retry -q


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [19]:
# Uninstall NLTK versi saat ini
!pip uninstall nltk -y -q

# Install NLTK versi 3.8.1 (versi stabil yang diketahui berfungsi)
!pip install nltk==3.8.1 -q

import nltk

# Download NLTK data
nltk.download('punkt')
nltk.download('punkt_tab')

# Lanjutkan dengan kode augmentasi lainnya


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


[nltk_data] Downloading package punkt to /Users/irfan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /Users/irfan/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [20]:
import pandas as pd
import re
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# === 1. Load data ===
df = pd.read_csv("gojek_news_preprocessed_before_augmentation.csv")

# === 2. Buat stemmer ===
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# === 3. Daftar kata kunci sentimen ===
positive_words = [
    "kenaikan pendapatan", "bonus", "subsidi", "inovasi", "promo", "diskon",
    "berhasil", "puas", "baik", "bagus", "pertumbuhan", "kerjasama", "investasi",
    "meningkat", "sukses", "memuaskan", "keuntungan"
]

negative_words = [
    "mogok", "tidak puas", "demo", "turun", "protes", "masalah", "gagal", "kecelakaan",
    "tarif naik", "kenaikan tarif", "komisi turun", "kerugian", "resign", "boikot", "keluhan",
    "keberatan", "penurunan", "merugi", "turunnya pendapatan", "tidak layak"
]

negation_words = ["tidak", "kurang", "belum", "bukan"]

# === 4. Fungsi labeling lebih kuat ===
def auto_label(text):
    text = str(text).lower()
    stemmed_text = stemmer.stem(text)

    # cek negasi + kata positif → negatif
    for neg in negation_words:
        for pos in positive_words:
            if f"{neg} {pos}" in stemmed_text:
                return "negatif"

    # cek kata positif & negatif
    if any(w in stemmed_text for w in positive_words):
        return "positif"
    elif any(w in stemmed_text for w in negative_words):
        return "negatif"
    else:
        return "netral"

# === 5. Terapkan ke kolom content ===
df["label"] = df["content"].astype(str).apply(auto_label)
df.to_csv("gojek_news_preprocessed_before_augmentation.csv", index=False)
print("✅ Label otomatis ditambahkan dengan deteksi negasi dan stemming.")


✅ Label otomatis ditambahkan dengan deteksi negasi dan stemming.


In [21]:
import pandas as pd
import re
import logging
from deep_translator import GoogleTranslator
from langdetect import detect, LangDetectException
import random
import time
from retry import retry
from nltk.tokenize import word_tokenize
import nltk
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from multiprocessing import Pool
import numpy as np

# === Download NLTK data ===
import nltk
nltk.data.path.append("/Users/irfan/nltk_data")

# === Inisialisasi logging ===
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# === Load file hasil preprocessing ===
try:
    df = pd.read_csv('gojek_news_preprocessed_before_augmentation.csv')
    logger.info(f"✅ Berhasil memuat file gojek_news_preprocessed_before_augmentation.csv dengan {len(df)} baris")
except FileNotFoundError:
    logger.error("❌ File gojek_news_preprocessed_before_augmentation.csv tidak ditemukan.")
    raise

# === Validasi kolom ===
required_columns = ['judul', 'cleaned_content', 'label']
if not all(col in df.columns for col in required_columns):
    missing_cols = [col for col in required_columns if col not in df.columns]
    logger.error(f"Kolom yang hilang: {missing_cols}")
    raise ValueError(f"DataFrame tidak memiliki kolom yang diperlukan: {missing_cols}")

# === Kata kunci utama GOJEK ===
gojek_keywords = [
    'gojek', 'grab', 'gopay', 'goride', 'gocar', 'gofood', 'gosend', 'goto',
    'driver', 'pengemudi', 'ojek online', 'ojol', 'transportasi', 'mitra',
    'aplikasi', 'layanan', 'subsidi', 'tarif', 'komisi', 'promo', 'startup',
    'investasi', 'digital', 'pelanggan', 'pesanan', 'antar jemput'
]

# === Inisialisasi translator dengan caching ===
translation_cache = {}
translator = GoogleTranslator(source='auto', target='en')
translator_id = GoogleTranslator(source='en', target='id')

# === Kamus sinonim untuk konteks Gojek ===
synonym_dict = {
    'gojek': ['ojol', 'aplikasi transportasi', 'startup ride hailing'],
    'goride': ['layanan ojek', 'motor online'],
    'gocar': ['layanan mobil', 'taksi online'],
    'gofood': ['layanan makanan', 'pesan makanan'],
    'gosend': ['layanan antar', 'kurir online'],
    'gopay': ['dompet digital', 'layanan pembayaran'],
    'driver': ['pengemudi', 'mitra', 'ojol'],
    'mitra': ['pengemudi', 'pekerja', 'rekan gojek'],
    'tarif': ['biaya', 'harga', 'ongkos'],
    'pelanggan': ['customer', 'pengguna', 'penumpang'],
    'layanan': ['fitur', 'jasa', 'produk'],
    'promo': ['diskon', 'potongan harga', 'penawaran'],
    'investasi': ['pendanaan', 'modal', 'suntikan dana'],
    'digital': ['online', 'virtual', 'elektronik'],
    'aplikasi': ['app', 'platform', 'software']
}

# === Inisialisasi stemmer ===
stemmer_factory = StemmerFactory()
stemmer = stemmer_factory.create_stemmer()

# === Fungsi penggantian sinonim ===
def synonym_replacement(text, n=3):
    try:
        words = word_tokenize(text)
        new_words = words.copy()
        indices = list(range(len(words)))
        random.shuffle(indices)
        replaced = 0
        for i in indices:
            word = words[i].lower()
            if word in synonym_dict:
                synonyms = [syn for syn in synonym_dict[word] if syn != word]
                if synonyms:
                    new_words[i] = random.choice(synonyms)
                    replaced += 1
            if replaced >= n:
                break
        return " ".join(new_words)
    except Exception as e:
        logger.error(f"Error in synonym_replacement: {e}")
        return text

# === Fungsi untuk parafrase sederhana ===
def simple_paraphrase(text):
    try:
        words = word_tokenize(text)
        if len(words) < 5:
            return text
        key_positions = [i for i, word in enumerate(words) if word.lower() in gojek_keywords]
        if len(key_positions) < 2:
            return text
        pos1, pos2 = random.sample(key_positions, 2)
        words[pos1], words[pos2] = words[pos2], words[pos1]
        return " ".join(words)
    except Exception as e:
        logger.error(f"Error in simple_paraphrase: {e}")
        return text

# === Fungsi pemotongan teks panjang ===
def truncate_text(text, max_length=5000):
    if len(text) > max_length:
        logger.warning(f"Teks terlalu panjang ({len(text)} karakter), dipotong jadi {max_length}.")
        return text[:max_length]
    return text

# === Fungsi back-translation (Indonesia → Inggris → Indonesia) ===
@retry(tries=3, delay=1, backoff=2)
def back_translate(text, intermediate_lang='en'):
    try:
        text = truncate_text(text)
        if text in translation_cache:
            return translation_cache[text]
        translated = GoogleTranslator(source='id', target=intermediate_lang).translate(text)
        time.sleep(0.5)
        back_translated = GoogleTranslator(source=intermediate_lang, target='id').translate(translated)
        time.sleep(0.5)
        translation_cache[text] = back_translated
        return back_translated
    except Exception as e:
        logger.error(f"Error in back-translation: {e}")
        return text

# === Validasi hasil augmentasi ===
def validate_augmented_text(text):
    try:
        if len(text.split()) < 5:
            logger.warning(f"Teks terlalu pendek: {text[:50]}...")
            return False
        if detect(text) != 'id':
            logger.warning(f"Teks bukan bahasa Indonesia: {text[:50]}...")
            return False
        keyword_count = sum(1 for keyword in gojek_keywords if keyword.lower() in text.lower())
        if keyword_count < 1:
            logger.warning(f"Teks kurang relevan (tidak mengandung cukup kata kunci Gojek): {text[:50]}...")
            return False
        return True
    except LangDetectException:
        logger.warning(f"Deteksi bahasa gagal: {text[:50]}...")
        return False
    except Exception as e:
        logger.error(f"Error in validate_augmented_text: {e}")
        return False

# === Membersihkan hasil augmentasi ===
def clean_augmented_text(text):
    try:
        text = re.sub(r'\s+', ' ', text).strip()
        text = re.sub(r'[^\w\s.,!?éê]', '', text)
        words = word_tokenize(text)
        stemmed_words = [stemmer.stem(word) if word.lower() not in gojek_keywords else word for word in words]
        return " ".join(stemmed_words)
    except Exception as e:
        logger.error(f"Error in clean_augmented_text: {e}")
        return text

# === Fungsi augmentasi satu baris ===
def augment_row(row, intermediate_langs):
    try:
        original_text = row['cleaned_content']
        method = random.choice(['back_translate', 'synonym', 'paraphrase'])

        if method == 'back_translate':
            intermediate_lang = random.choice(intermediate_langs)
            augmented_text = back_translate(original_text, intermediate_lang=intermediate_lang)
        elif method == 'synonym':
            augmented_text = synonym_replacement(original_text, n=3)
        else:
            augmented_text = simple_paraphrase(original_text)

        augmented_text = clean_augmented_text(augmented_text)
        if validate_augmented_text(augmented_text):
            row['cleaned_content'] = augmented_text
            row['is_augmented'] = True
            return row
        return None
    except Exception as e:
        logger.error(f"Error in augment_row: {e}")
        return None

# === Parameter augmentasi ===
TARGET_PER_LABEL = 100
INTERMEDIATE_LANGS = ['en', 'fr', 'de']
dfs = []

logger.info("🚀 Mulai proses augmentasi per label...")

for label in df['label'].unique():
    subset = df[df['label'] == label].copy()
    count = len(subset)
    logger.info(f"Label '{label}' memiliki {count} data")

    if count >= TARGET_PER_LABEL:
        sampled = subset.sample(n=TARGET_PER_LABEL, random_state=42).copy()
        sampled['is_augmented'] = False
        dfs.append(sampled)
    else:
        subset['is_augmented'] = False
        dfs.append(subset)
        n_to_augment = TARGET_PER_LABEL - count
        logger.info(f"Label '{label}': Perlu augmentasi {n_to_augment} data")

        rows_to_augment = [subset.iloc[random.randint(0, count-1)].copy() for _ in range(n_to_augment * 2)]
        results = [augment_row(row, INTERMEDIATE_LANGS) for row in rows_to_augment]


        augmented_rows = [row for row in results if row is not None][:n_to_augment]

        if len(augmented_rows) < n_to_augment:
            logger.warning(f"Label '{label}': Hanya menghasilkan {len(augmented_rows)} dari {n_to_augment} data. Menambah data asli.")
            remaining = n_to_augment - len(augmented_rows)
            for i in range(remaining):
                base_row = subset.iloc[random.randint(0, count-1)].copy()
                base_row['is_augmented'] = False
                augmented_rows.append(base_row)

        df_aug = pd.DataFrame(augmented_rows)
        dfs.append(df_aug)
        logger.info(f"Label '{label}': Total data setelah augmentasi: {len(augmented_rows) + count}")

# === Gabungkan semua hasil ===
df_final = pd.concat(dfs, ignore_index=True)

# === Simpan hasil ===
df_final.to_csv('gojek_news_augmented_full_pipeline.csv', index=False)
df_final.to_excel('gojek_news_augmented_full_pipeline.xlsx', index=False)
logger.info("✅ Proses augmentasi selesai, data disimpan ke gojek_news_augmented_full_pipeline.csv dan .xlsx")

# === Ringkasan akhir ===
logger.info(f"Total data akhir: {len(df_final)}")
logger.info(f"Distribusi label:\n{df_final['label'].value_counts().to_string()}")
logger.info("Sampel 5 baris pertama setelah augmentasi:")
logger.info(df_final[['judul', 'cleaned_content', 'label', 'is_augmented']].head().to_string())


2025-10-28 16:28:38,182 - INFO - ✅ Berhasil memuat file gojek_news_preprocessed_before_augmentation.csv dengan 143 baris
2025-10-28 16:28:38,202 - INFO - 🚀 Mulai proses augmentasi per label...
2025-10-28 16:28:38,228 - INFO - Label 'negatif' memiliki 24 data
2025-10-28 16:28:38,234 - INFO - Label 'negatif': Perlu augmentasi 76 data
2025-10-28 16:28:50,410 - WARNING - Teks bukan bahasa Indonesia: gojek indonesia has engaged and communicated to dr...
2025-10-28 16:28:59,150 - WARNING - Teks bukan bahasa Indonesia: gojek indonesia has engaged and communicated to dr...
2025-10-28 16:29:38,441 - WARNING - Teks bukan bahasa Indonesia: gojek indonesia has engaged and communicated to dr...
2025-10-28 16:29:38,448 - WARNING - Teks bukan bahasa Indonesia: gojek indonesia has engaged and communicated to dr...
2025-10-28 16:29:42,482 - WARNING - Teks bukan bahasa Indonesia: gojek indonesia has engaged and communicated to dr...
2025-10-28 16:29:52,111 - WARNING - Teks bukan bahasa Indonesia: gojek 